In [38]:
import csv
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer as CV
import nltk
import re
from IPython.display import display, HTML

In [2]:
with open('stored_variables/sfp_subreddits_by_author.json', 'r') as f:
    sfp_subreddits_by_author = json.load(f)

with open('stored_variables/td_subreddits_by_author.json', 'r') as f:
    td_subreddits_by_author = json.load(f)
    
with open('stored_variables/neither_subreddits_by_author.json', 'r') as f:
    neither_subreddits_by_author = json.load(f)

In [3]:
political_sfp_lst = []

for k in sfp_subreddits_by_author.keys():
    if 'politics' in sfp_subreddits_by_author[k].keys():
        political_sfp_lst.append(sfp_subreddits_by_author[k]['politics'])
        
print(sum(political_sfp_lst))
print(len(political_sfp_lst))

political_td_lst = []

for k in td_subreddits_by_author.keys():
    if 'politics' in td_subreddits_by_author[k].keys():
        political_td_lst.append(td_subreddits_by_author[k]['politics'])
        
print(sum(political_td_lst))
print(len(political_td_lst))

political_neither_lst = []

for k in neither_subreddits_by_author.keys():
    if 'politics' in neither_subreddits_by_author[k].keys():
        political_neither_lst.append(neither_subreddits_by_author[k]['politics'])
        
print(sum(political_neither_lst))
print(len(political_neither_lst))

115414
1289
65378
1087
7971
346


In [4]:
sfp_data = pd.read_csv("../sample_data/sfp_langsample_v4.csv")

In [5]:
td_data = pd.read_csv("../sample_data/td_langsample_v4.csv")

In [51]:
politics_data = pd.read_csv("../sample_data/politics_sample3.csv", 
                            header=None, 
                            names = ['author', 'subreddit', 'body', 'score', 'created_dt', 'link_id', 'parent_id'],
                            index_col = False)

In [8]:
print(sfp_data.columns)
print(td_data.columns)
print(politics_data.columns)

Index(['author', 'subreddit', 'body', 'created_utc', 'created_dt'], dtype='object')
Index(['author', 'subreddit', 'body', 'created_utc', 'created_dt'], dtype='object')
Index(['author', 'subreddit', 'body', 'score', 'created_dt', 'rand'], dtype='object')


In [52]:
politics_text = politics_data.loc[politics_data['body'].notna()]['body'].values

In [17]:
sfp_authors = sfp_data['author'].unique()
td_authors = td_data['author'].unique()
print(len(sfp_authors))
print(len(td_authors))

2000
2000


In [18]:
print(np.sum(sfp_data['subreddit'] == 'SandersForPresident'))
print(np.sum(sfp_data['subreddit'] == 'The_Donald'))
print(np.sum(td_data['subreddit'] == 'SandersForPresident'))
print(np.sum(td_data['subreddit'] == 'The_Donald'))

82916
1885
4476
62877


In [19]:
sfp_posts = sfp_data.loc[sfp_data['subreddit'] == 'SandersForPresident']
td_posts = td_data.loc[td_data['subreddit'] == 'The_Donald']

In [20]:
min_overall_time = 1420131600
max_overall_time = 1464753600

In [21]:
sfp_min_times = [min(sfp_posts.loc[sfp_posts['author'] == auth]['created_utc']) for auth in sfp_authors]

In [22]:
td_min_times = [min(td_posts.loc[td_posts['author'] == auth]['created_utc']) for auth in td_authors]

In [23]:
sfp_politics = sfp_data.loc[sfp_data['subreddit'] == 'politics']
td_politics = td_data.loc[td_data['subreddit'] == 'politics']

sfp_politics = sfp_politics.loc[sfp_politics['created_utc'] >= min_overall_time]
sfp_politics = sfp_politics.loc[sfp_politics['created_utc'] < max_overall_time]

td_politics = td_politics.loc[td_politics['created_utc'] >= min_overall_time]
td_politics = td_politics.loc[td_politics['created_utc'] < max_overall_time]

print(sfp_politics.shape)
print(td_politics.shape)

(100042, 5)
(35702, 5)


In [25]:
print(sum(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]]['created_utc'] < sfp_min_times[0]))
print(sum(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]]['created_utc'] > sfp_min_times[0]))

3
26


In [26]:
len(sfp_politics)

100042

In [27]:
sfp_politics_before = pd.DataFrame([], columns = sfp_politics.columns)
sfp_politics_after = pd.DataFrame([], columns = sfp_politics.columns)

for i in range(len(sfp_authors)):
    curr_auth_posts = sfp_politics.loc[sfp_politics['author'] == sfp_authors[i]]
    
    if len(curr_auth_posts) > 0:
        sfp_politics_before = \
sfp_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < sfp_min_times[i]])
        
        sfp_politics_after = \
sfp_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > sfp_min_times[i]])
    

In [28]:
sfp_politics_before

,author,subreddit,body,created_utc,created_dt
3141065,english_guy_78,politics,"Yes, he wants to introduce more tax brackets a...",1436539325,2015-07-10 10:42:05-04
3141436,english_guy_78,politics,Taxes are already the same for everyone - ever...,1436544840,2015-07-10 12:14:00-04
3142125,english_guy_78,politics,I'd prefer we have a very progressive system o...,1436555110,2015-07-10 15:05:10-04
2944167,thinkbox,politics,Proposing something during election season? \n...,1431638003,2015-05-14 17:13:23-04
3178064,thinkbox,politics,"""Brogressive""\n\nI haven't heard this term bef...",1437441787,2015-07-20 21:23:07-04
...,...,...,...,...,...
3916120,DeserveDonuts,politics,"Oh sorry, I mixed it up with benzos for some r...",1453101565,2016-01-18 02:19:25-05
3916177,DeserveDonuts,politics,"I am that person, and I've never sold or given...",1453102670,2016-01-18 02:37:50-05
3916277,DeserveDonuts,politics,&gt;you're supposed to be taking them steadily...,1453105737,2016-01-18 03:28:57-05
3315277,_IAlwaysLie,politics,"It's the ""You realize, right?"" It builds up th...",1440562655,2015-08-26 00:17:35-04


In [29]:
sfp_politics_after

,author,subreddit,body,created_utc,created_dt
3178299,english_guy_78,politics,"""The legislation would raise the minimum wage ...",1437446254,2015-07-20 22:37:34-04
3178330,english_guy_78,politics,I don't think this really adds up. Who would b...,1437446891,2015-07-20 22:48:11-04
3178426,english_guy_78,politics,Isn't that an argument FOR a higher minimum wa...,1437448905,2015-07-20 23:21:45-04
3178459,english_guy_78,politics,But your initial point was that without Walmar...,1437449647,2015-07-20 23:34:07-04
3178477,english_guy_78,politics,No your point was that these workers would be ...,1437450137,2015-07-20 23:42:17-04
...,...,...,...,...,...
4480955,DeserveDonuts,politics,'minds on drugs' or something like that :J,1463071631,2016-05-12 12:47:11-04
4527121,DeserveDonuts,politics,No that's Larry Garfield,1464040011,2016-05-23 17:46:51-04
4291577,_IAlwaysLie,politics,"I think the answer is nationally-funded, not p...",1459575788,2016-04-02 01:43:08-04
4304296,_IAlwaysLie,politics,Sanders estimates that the raised taxes will b...,1459820217,2016-04-04 21:36:57-04


In [30]:
td_politics_before = pd.DataFrame([], columns = td_politics.columns)
td_politics_after = pd.DataFrame([], columns = td_politics.columns)

for i in range(len(td_authors)):
    curr_auth_posts = td_politics.loc[td_politics['author'] == td_authors[i]]
    
    if len(curr_auth_posts) > 0:
        td_politics_before = \
td_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < td_min_times[i]])
        
        td_politics_after = \
td_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > td_min_times[i]])
    

In [31]:
td_politics_before

,author,subreddit,body,created_utc,created_dt
2187850,berlinbrown,politics,"""make up the difference between what you pay a...",1426732004,2015-03-18 22:26:44-04
2187867,berlinbrown,politics,I have done so many jobs in my lifetime in dif...,1426732386,2015-03-18 22:33:06-04
2187892,berlinbrown,politics,But there is still no law on who they can hire...,1426732887,2015-03-18 22:41:27-04
2187909,berlinbrown,politics,One might argue and most things don't operate ...,1426733133,2015-03-18 22:45:33-04
2187930,berlinbrown,politics,I am not making any assumptions. Why should t...,1426733393,2015-03-18 22:49:53-04
...,...,...,...,...,...
3226400,MyWifeDontKnowItsMe,politics,How about you show me evidence that prohibitin...,1452132743,2016-01-06 21:12:23-05
3230507,MyWifeDontKnowItsMe,politics,That's the claim you have been making this ent...,1452217872,2016-01-07 20:51:12-05
3236723,MyWifeDontKnowItsMe,politics,You claimed that ownership and investment are ...,1452375380,2016-01-09 16:36:20-05
3304943,MyWifeDontKnowItsMe,politics,Now I'm definitely voting for Cruz.,1453938614,2016-01-27 18:50:14-05


In [32]:
td_politics_after

,author,subreddit,body,created_utc,created_dt
3442549,berlinbrown,politics,Trump is an amazing showman. He is winning be...,1457057276,2016-03-03 21:07:56-05
3442637,berlinbrown,politics,"Kasich giving no fucks, he is just putting him...",1457058611,2016-03-03 21:30:11-05
3442642,berlinbrown,politics,"OH OK, no clap",1457058721,2016-03-03 21:32:01-05
3442648,berlinbrown,politics,I think they are dancing for the camera.,1457058824,2016-03-03 21:33:44-05
3442668,berlinbrown,politics,"""Trump, Trump, fucking gave to Clinton for bus...",1457059105,2016-03-03 21:38:25-05
...,...,...,...,...,...
3793833,MyWifeDontKnowItsMe,politics,"If that's something you are just dying to see,...",1464254114,2016-05-26 05:15:14-04
3793838,MyWifeDontKnowItsMe,politics,Let's all put this notion to rest that Donald ...,1464254310,2016-05-26 05:18:30-04
3793840,MyWifeDontKnowItsMe,politics,I'm guessing they may have screened people.,1464254382,2016-05-26 05:19:42-04
3793861,MyWifeDontKnowItsMe,politics,"Or, you know, check voter registration, social...",1464255369,2016-05-26 05:36:09-04


In [33]:
#sanity check
display(sfp_politics.loc[sfp_politics['author'] == sfp_authors[0]])
display(sfp_posts.loc[sfp_data['author'] == sfp_authors[0]])
display(sfp_politics_before.loc[sfp_politics_before['author'] == sfp_authors[0]])
display(sfp_politics_after.loc[sfp_politics_after['author'] == sfp_authors[0]])

,author,subreddit,body,created_utc,created_dt
3141065,english_guy_78,politics,"Yes, he wants to introduce more tax brackets a...",1436539325,2015-07-10 10:42:05-04
3141436,english_guy_78,politics,Taxes are already the same for everyone - ever...,1436544840,2015-07-10 12:14:00-04
3142125,english_guy_78,politics,I'd prefer we have a very progressive system o...,1436555110,2015-07-10 15:05:10-04
3178299,english_guy_78,politics,"""The legislation would raise the minimum wage ...",1437446254,2015-07-20 22:37:34-04
3178330,english_guy_78,politics,I don't think this really adds up. Who would b...,1437446891,2015-07-20 22:48:11-04
3178426,english_guy_78,politics,Isn't that an argument FOR a higher minimum wa...,1437448905,2015-07-20 23:21:45-04
3178459,english_guy_78,politics,But your initial point was that without Walmar...,1437449647,2015-07-20 23:34:07-04
3178477,english_guy_78,politics,No your point was that these workers would be ...,1437450137,2015-07-20 23:42:17-04
3178516,english_guy_78,politics,If you don't believe that Walmart creates the ...,1437450855,2015-07-20 23:54:15-04
3178525,english_guy_78,politics,Surely this just highlights that the other job...,1437450969,2015-07-20 23:56:09-04


,author,subreddit,body,created_utc,created_dt
3159698,english_guy_78,SandersForPresident,$50/mo here. Just threw an extra $50 in as we...,1436987828,2015-07-15 15:17:08-04
3162437,english_guy_78,SandersForPresident,It's about spending the money in the right pla...,1437060156,2015-07-16 11:22:36-04
3427212,english_guy_78,SandersForPresident,Party affiliation: democrat + independent = Hi...,1443055183,2015-09-23 20:39:43-04
4084138,english_guy_78,SandersForPresident,"If the entrance polls are correct, Clinton nee...",1456002132,2016-02-20 16:02:12-05
4084157,english_guy_78,SandersForPresident,"I'm not saying she can't. I'm just saying, lo...",1456002379,2016-02-20 16:06:19-05
...,...,...,...,...,...
4412476,english_guy_78,SandersForPresident,It's irrelevant it's about fracking. It's the...,1461714531,2016-04-26 19:48:51-04
4412481,english_guy_78,SandersForPresident,It's irrelevant it's about fracking. It's the ...,1461714557,2016-04-26 19:49:17-04
4443761,english_guy_78,SandersForPresident,Watching on MSNBC. They talk all day about co...,1462324418,2016-05-03 21:13:38-04
4503393,english_guy_78,SandersForPresident,Anyone have exit polls? It's pretty interesti...,1463534519,2016-05-17 21:21:59-04


,author,subreddit,body,created_utc,created_dt
3141065,english_guy_78,politics,"Yes, he wants to introduce more tax brackets a...",1436539325,2015-07-10 10:42:05-04
3141436,english_guy_78,politics,Taxes are already the same for everyone - ever...,1436544840,2015-07-10 12:14:00-04
3142125,english_guy_78,politics,I'd prefer we have a very progressive system o...,1436555110,2015-07-10 15:05:10-04


,author,subreddit,body,created_utc,created_dt
3178299,english_guy_78,politics,"""The legislation would raise the minimum wage ...",1437446254,2015-07-20 22:37:34-04
3178330,english_guy_78,politics,I don't think this really adds up. Who would b...,1437446891,2015-07-20 22:48:11-04
3178426,english_guy_78,politics,Isn't that an argument FOR a higher minimum wa...,1437448905,2015-07-20 23:21:45-04
3178459,english_guy_78,politics,But your initial point was that without Walmar...,1437449647,2015-07-20 23:34:07-04
3178477,english_guy_78,politics,No your point was that these workers would be ...,1437450137,2015-07-20 23:42:17-04
3178516,english_guy_78,politics,If you don't believe that Walmart creates the ...,1437450855,2015-07-20 23:54:15-04
3178525,english_guy_78,politics,Surely this just highlights that the other job...,1437450969,2015-07-20 23:56:09-04
3178558,english_guy_78,politics,But then by your logic ALL employers regardles...,1437451498,2015-07-21 00:04:58-04
3178582,english_guy_78,politics,Clearly it follows pay for all jobs will incre...,1437451804,2015-07-21 00:10:04-04
3178632,english_guy_78,politics,Pay must increase to motivate people to do har...,1437452766,2015-07-21 00:26:06-04


In [34]:
sfp_politics_before_text = sfp_politics_before.loc[sfp_politics_before['body'].notna()]['body'].values
sfp_politics_after_text = sfp_politics_after.loc[sfp_politics_after['body'].notna()]['body'].values

td_politics_before_text = td_politics_before.loc[td_politics_before['body'].notna()]['body'].values
td_politics_after_text = td_politics_after.loc[td_politics_after['body'].notna()]['body'].values

sfp_posts_text = sfp_posts.loc[sfp_posts['body'].notna()]['body'].values
td_posts_text = td_posts.loc[td_posts['body'].notna()]['body'].values

In [35]:
def tokenize_data(data, deleted_str = '[deleted]', removed_str = '[removed]'):
    print("Number of entries in original dataset: " + str(len(data)))
    data1 = [i for i in data if 'thank you for participating' not in i.lower()]
    print("Number of moderator posts removed from dataset: " + str(len([i for i in data if 'thank you for participating' in i.lower()])))
    data1 = [i for i in data1 if i != deleted_str and i != removed_str]
    print("Number of user-deleted posts removed from dataset: " + str(len([i for i in data if i == deleted_str])))
    print("Number of mod/admin-deleted posts removed from dataset: " + str(len([i for i in data if i == removed_str])))
    
    print("Number of entries in modified dataset: " + str(len(data1)))
    
    # removing hyperlinks
    print("Number of comments with a hyperlink in modified dataset: " + str(len([i for i in data1 if "http" in i])))
    data1 = [re.sub(r'https?:\/\/[\S]+', ' ', i, flags=re.MULTILINE) for i in data1]
    
    # common escape sequences
    data1 = [re.sub(r'\n', '', i) for i in data1]
    data1 = [re.sub(r'&gt;', '', i) for i in data1]
    
    # convert to lowercase
    data1 = [i.lower() for i in data1]
    
    # keep user and subreddit tags
    print("Number of comments that mention a user in the modified dataset: " + str(len([i for i in data1 if "/u/" in i])))
    print("Number of comments that mention a subreddit in the modified dataset: " + str(len([i for i in data1 if "/r/" in i])))
    data1 = [re.sub(r'/u/', '_user_', i) for i in data1]
    data1 = [re.sub(r'/r/', '_subreddit_', i) for i in data1]
    
    data_tokenized = [re.findall(r'\w+', i.lower()) for i in data1]
    
    print("Number of total tokens: " + str(sum([len(d) for d in data_tokenized])))
    
    return [(' '.join(i) + '\n') for i in data_tokenized]


In [54]:
filenames = ['politics_train.txt', 
             'sfp_politics_before.txt', 
             'sfp_politics_after.txt',
             'td_politics_before.txt',
             'td_politics_after.txt',
             'sfp_posts.txt',
             'td_posts.txt']

text_arrs = [politics_text,
             sfp_politics_before_text,
             sfp_politics_after_text,
             td_politics_before_text,
             td_politics_after_text,
             sfp_posts_text,
             td_posts_text]

train_arrs = []

for i in range(len(text_arrs)):
    lm_train_arr = tokenize_data(text_arrs[i])
    print()
    train_arrs.append(lm_train_arr)

Number of entries in original dataset: 999960
Number of moderator posts removed from dataset: 7782
Number of user-deleted posts removed from dataset: 53033
Number of mod/admin-deleted posts removed from dataset: 13208
Number of entries in modified dataset: 925937
Number of comments with a hyperlink in modified dataset: 52354
Number of comments that mention a user in the modified dataset: 1993
Number of comments that mention a subreddit in the modified dataset: 7655
Number of total tokens: 38189047

Number of entries in original dataset: 21035
Number of moderator posts removed from dataset: 0
Number of user-deleted posts removed from dataset: 0
Number of mod/admin-deleted posts removed from dataset: 0
Number of entries in modified dataset: 21035
Number of comments with a hyperlink in modified dataset: 1066
Number of comments that mention a user in the modified dataset: 39
Number of comments that mention a subreddit in the modified dataset: 203
Number of total tokens: 864027

Number of e

In [41]:
for i in range(len(train_arrs)):
    print(train_arrs[i][:10])
    print("\n\n")

['true but the only way that is likely to occur is to start rubbing the general public s noses in the way that the 1 have already stolen most of their country\n', 'fuck him right bernie or bust\n', 'cnn did a map breakdown last night where they gave bernie a win in every remaining state and guess what\n', '\n', 'i disagree the entire race basically every mainstream media outlet has shown the delegate count including super delegates giving clinton the appearance of being like over 4 5 hundred delegates in the lead when it s much closer to like 100 250 you may think this has no impact but i would imagine there is some slight impact based on this some studies have shown that voters are much more inclined to choose a candidate if they believe they are definitely going to win the nomination the fact that the msm has been saying since day one bernie has no chance and including super delegates in their counts it s made it appear less close than it actually is that being said i think hilary wo

In [55]:
for i in range(len(train_arrs)):
    lm_train_file = open(filenames[i], 'w+')
    lm_train_file.writelines(train_arrs[i])
    lm_train_file.close()

In [56]:
for i in range(len(train_arrs)):
    print(len(train_arrs[i]))

925937
21035
78953
20131
15571
82545
62862
